In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

<H1> Pre - Processamento </H1>

In [2]:
data = np.loadtxt('DATA.txt')
data = pd.DataFrame(data, columns=['gender', 'age', 'risk_factors', 'systolic', 'HR_sbp', 'st', 'HR_ecg', 'creatinine', 'killip', 'event'])

data['gender'] = data['gender'].astype('int')
data['risk_factors'] = data['risk_factors'].astype('int')
data['st'] = data['st'].astype('int')
data['killip'] = data['killip'].astype('int')
data['event'] = data['event'].astype('int')

data

,gender,age,risk_factors,systolic,HR_sbp,st,HR_ecg,creatinine,killip,event
0,1,33.0,0,132.0,91.579173,1,90.0,0.8,1,0
1,1,69.0,0,147.0,52.785861,0,52.0,1.4,1,0
2,1,63.0,0,142.0,40.692798,1,44.0,1.1,3,1
3,0,79.0,0,147.0,106.420820,1,110.0,0.9,1,1
4,0,61.0,0,107.0,82.148302,0,80.0,1.1,1,0
...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,99.801711,0,103.0,1.6,3,1
453,1,51.0,1,140.0,87.339759,1,90.0,1.0,3,1
454,0,57.0,0,120.0,113.138910,0,117.0,1.0,3,1
455,1,87.0,0,149.0,50.035793,1,53.0,1.4,1,1


<H1> Fusão de Sensores </H1>
<p> Deviation</p>
<p>    (HRBP)=2</p>
<p>    (HRECG)=0.5 </p>

In [3]:
HR_sbp_std = 2
HR_ecg_std = 0.5

data['HR'] = np.power(HR_ecg_std,2) / (np.power(HR_sbp_std,2) + np.power(HR_ecg_std,2)) * data['HR_sbp'] + np.power(HR_sbp_std,2) / (np.power(HR_sbp_std,2) + np.power(HR_ecg_std,2)) * data['HR_ecg']

data

,gender,age,risk_factors,systolic,HR_sbp,st,HR_ecg,creatinine,killip,event,HR
0,1,33.0,0,132.0,91.579173,1,90.0,0.8,1,0,90.092893
1,1,69.0,0,147.0,52.785861,0,52.0,1.4,1,0,52.046227
2,1,63.0,0,142.0,40.692798,1,44.0,1.1,3,1,43.805459
3,0,79.0,0,147.0,106.420820,1,110.0,0.9,1,1,109.789460
4,0,61.0,0,107.0,82.148302,0,80.0,1.1,1,0,80.126371
...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,99.801711,0,103.0,1.6,3,1,102.811865
453,1,51.0,1,140.0,87.339759,1,90.0,1.0,3,1,89.843515
454,0,57.0,0,120.0,113.138910,0,117.0,1.0,3,1,116.772877
455,1,87.0,0,149.0,50.035793,1,53.0,1.4,1,1,52.825635


In [4]:
data['guidelines'] = np.where(((data["creatinine"] >= 1.3) & (data["st"] == 1)) | (data["killip"] >= 2), 1, 0)
Event_0 = data[data['event'] == 0]
Event_1 = data[data['event'] == 1]


display(data)

,gender,age,risk_factors,systolic,HR_sbp,st,HR_ecg,creatinine,killip,event,HR,guidelines
0,1,33.0,0,132.0,91.579173,1,90.0,0.8,1,0,90.092893,0
1,1,69.0,0,147.0,52.785861,0,52.0,1.4,1,0,52.046227,0
2,1,63.0,0,142.0,40.692798,1,44.0,1.1,3,1,43.805459,1
3,0,79.0,0,147.0,106.420820,1,110.0,0.9,1,1,109.789460,0
4,0,61.0,0,107.0,82.148302,0,80.0,1.1,1,0,80.126371,0
...,...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,99.801711,0,103.0,1.6,3,1,102.811865,1
453,1,51.0,1,140.0,87.339759,1,90.0,1.0,3,1,89.843515,1
454,0,57.0,0,120.0,113.138910,0,117.0,1.0,3,1,116.772877,1
455,1,87.0,0,149.0,50.035793,1,53.0,1.4,1,1,52.825635,1


<H1> Valores Discretos </H1>
<p>-> gender</p>
<p>-> risk_factors</p>
<p>-> st_segment </p>
<p>-> killip</p>
<p>----> Slides BayesianDecision 32 <----- </p>

In [5]:
P_gender_0 = Event_0['gender'].value_counts() / Event_0.shape[0]
P_gender_1 = Event_1['gender'].value_counts() / Event_1.shape[0]

P_risk_0 = Event_0['risk_factors'].value_counts() / Event_0.shape[0]
P_risk_1 = Event_1['risk_factors'].value_counts() / Event_1.shape[0]

P_st_0 = Event_0['st'].value_counts() / Event_0.shape[0]
P_st_1 = Event_1['st'].value_counts() / Event_1.shape[0]

P_kill_0 = Event_0['killip'].value_counts() / Event_0.shape[0]
P_kill_1 = Event_1['killip'].value_counts() / Event_0.shape[0]

P_guide_0 = Event_0['guidelines'].value_counts() / Event_0.shape[0]
P_guide_1 = Event_1['guidelines'].value_counts() / Event_1.shape[0]

P_discrete = P_gender_1[data['gender']].to_numpy() * P_risk_1[data['risk_factors']].to_numpy() * P_st_1[data['st']].to_numpy() * P_kill_1[data['killip']].to_numpy() * P_guide_1[data['guidelines']].to_numpy()

P_not_discrete = P_gender_0[data['gender']].to_numpy()* P_risk_0[data['risk_factors']].to_numpy() * P_st_0[data['st']].to_numpy() * P_kill_0[data['killip']].to_numpy() * P_guide_0[data['guidelines']].to_numpy()

data['Disc_Val'] = P_discrete > P_not_discrete
data['Disc_Val'] = data['Disc_Val'].astype(int)
display(data)

,gender,age,risk_factors,systolic,HR_sbp,st,HR_ecg,creatinine,killip,event,HR,guidelines,Disc_Val
0,1,33.0,0,132.0,91.579173,1,90.0,0.8,1,0,90.092893,0,0
1,1,69.0,0,147.0,52.785861,0,52.0,1.4,1,0,52.046227,0,0
2,1,63.0,0,142.0,40.692798,1,44.0,1.1,3,1,43.805459,1,1
3,0,79.0,0,147.0,106.420820,1,110.0,0.9,1,1,109.789460,0,0
4,0,61.0,0,107.0,82.148302,0,80.0,1.1,1,0,80.126371,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,99.801711,0,103.0,1.6,3,1,102.811865,1,1
453,1,51.0,1,140.0,87.339759,1,90.0,1.0,3,1,89.843515,1,1
454,0,57.0,0,120.0,113.138910,0,117.0,1.0,3,1,116.772877,1,1
455,1,87.0,0,149.0,50.035793,1,53.0,1.4,1,1,52.825635,1,1


<H1> Valores Continuos </H1>
<p>-> age</p>
<p>-> systolic</p>
<p>-> hr </p>
<p>-> creatinine</p>
<p> ----> Slides BayesianDecision 47 <---- </p>

In [6]:
age_mean_0 = Event_0['age'].mean()
age_mean_1 = Event_1['age'].mean()
age_std_0 =  Event_0['age'].std()
age_std_1 =  Event_1['age'].std()

P_age_0 = 1 / (age_std_0 * np.sqrt(2 * np.pi)) * np.exp(-np.power(data['age'] - age_mean_0,2)/(2 * age_std_0**2))
P_age_1 = 1 / (age_std_1 * np.sqrt(2 * np.pi)) * np.exp(-np.power(data['age'] - age_mean_1,2)/(2 * age_std_1**2))

systolic_mean_0 = Event_0['systolic'].mean()
systolic_mean_1 = Event_1['systolic'].mean()
systolic_std_0 =  Event_0['systolic'].std()
systolic_std_1 =  Event_1['systolic'].std()

P_sys_0 = 1 / (systolic_std_0 * np.sqrt(2 * np.pi)) * np.exp(-np.power(data['systolic'] - systolic_mean_0,2)/(2 * systolic_std_0**2))
P_sys_1 = 1 / (systolic_std_1 * np.sqrt(2 * np.pi)) * np.exp(-np.power(data['systolic'] - systolic_mean_1,2)/(2 * systolic_std_1**2))

hr_mean_0 = Event_0['HR'].mean()
hr_mean_1 = Event_1['HR'].mean()
hr_std_0 =  Event_0['HR'].std()
hr_std_1 =  Event_1['HR'].std()

P_hr_0 = 1 / (hr_std_0 * np.sqrt(2 * np.pi)) * np.exp(-np.power(data['HR'] - hr_mean_0,2)/(2 * hr_std_0**2))
P_hr_1 = 1 / (hr_std_1 * np.sqrt(2 * np.pi)) * np.exp(-np.power(data['HR'] - hr_mean_1,2)/(2 * hr_std_1**2))

creatinine_mean_0 = Event_0['creatinine'].mean()
creatinine_mean_1 = Event_1['creatinine'].mean()
creatinine_std_0 =  Event_0['creatinine'].std()
creatinine_std_1 =  Event_1['creatinine'].std()

P_creatinine_0 = 1 / (creatinine_std_0 * np.sqrt(2 * np.pi)) * np.exp(-np.power(data['creatinine'] - creatinine_mean_0,2)/(2 * creatinine_std_0**2))
P_creatinine_1 = 1 / (creatinine_std_1 * np.sqrt(2 * np.pi)) * np.exp(-np.power(data['creatinine'] - creatinine_mean_1,2)/(2 * creatinine_std_1**2))

P_continuous = np.array(P_age_1 * P_sys_1* P_creatinine_1 * P_hr_1)
P_not_continuous = np.array(P_age_0 * P_sys_0 * P_creatinine_0 * P_hr_0)

data['Cont_Val'] = P_continuous > P_not_continuous
data['Cont_Val'] = data['Cont_Val'].astype(int)
display(data)

,gender,age,risk_factors,systolic,HR_sbp,st,HR_ecg,creatinine,killip,event,HR,guidelines,Disc_Val,Cont_Val
0,1,33.0,0,132.0,91.579173,1,90.0,0.8,1,0,90.092893,0,0,0
1,1,69.0,0,147.0,52.785861,0,52.0,1.4,1,0,52.046227,0,0,0
2,1,63.0,0,142.0,40.692798,1,44.0,1.1,3,1,43.805459,1,1,0
3,0,79.0,0,147.0,106.420820,1,110.0,0.9,1,1,109.789460,0,0,1
4,0,61.0,0,107.0,82.148302,0,80.0,1.1,1,0,80.126371,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,99.801711,0,103.0,1.6,3,1,102.811865,1,1,1
453,1,51.0,1,140.0,87.339759,1,90.0,1.0,3,1,89.843515,1,1,0
454,0,57.0,0,120.0,113.138910,0,117.0,1.0,3,1,116.772877,1,1,1
455,1,87.0,0,149.0,50.035793,1,53.0,1.4,1,1,52.825635,1,1,1


<H1> Análise </H1>
<p> ----> Slide FeatureFusion 36 <---- </p>

In [7]:
P_Positivos = P_discrete * P_continuous * (Event_1.shape[0] / data.shape[0])
P_Negativos = P_not_discrete * P_not_continuous * (Event_0.shape[0] / data.shape[0])

data['final'] = P_Positivos > P_Negativos
data['final'] = data['final'].astype(int)

display(data)

,gender,age,risk_factors,systolic,HR_sbp,st,HR_ecg,creatinine,killip,event,HR,guidelines,Disc_Val,Cont_Val,final
0,1,33.0,0,132.0,91.579173,1,90.0,0.8,1,0,90.092893,0,0,0,0
1,1,69.0,0,147.0,52.785861,0,52.0,1.4,1,0,52.046227,0,0,0,0
2,1,63.0,0,142.0,40.692798,1,44.0,1.1,3,1,43.805459,1,1,0,1
3,0,79.0,0,147.0,106.420820,1,110.0,0.9,1,1,109.789460,0,0,1,1
4,0,61.0,0,107.0,82.148302,0,80.0,1.1,1,0,80.126371,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,99.801711,0,103.0,1.6,3,1,102.811865,1,1,1,1
453,1,51.0,1,140.0,87.339759,1,90.0,1.0,3,1,89.843515,1,1,0,1
454,0,57.0,0,120.0,113.138910,0,117.0,1.0,3,1,116.772877,1,1,1,1
455,1,87.0,0,149.0,50.035793,1,53.0,1.4,1,1,52.825635,1,1,1,1


In [8]:
#Caso Binario
tn,fp,fn,tp = confusion_matrix(data['event'],data['final']).ravel()

sensibility = tp/(tp+fn)
print(sensibility)

0.6145251396648045


In [9]:
specifity = tn / (tn + fp)
print(specifity)

0.920863309352518


In [10]:
accuracy = (tp+tn) / (tp+tn+fp+fn)
print(accuracy)

0.8008752735229759


In [12]:
precision = tp/(tp+fp)
print(precision)

0.8333333333333334


In [15]:
print("Sensibility -->",sensibility)
print("Specifity -->",specifity)
print("Accuracy -->",accuracy)
print("Precision -->",precision)

Sensibility --> 0.6145251396648045
Specifity --> 0.920863309352518
Accuracy --> 0.8008752735229759
Precision --> 0.8333333333333334
